[← Modules](../../../getting_started/theory_to_python/modules.rst)

# DynamicalSystem

The `DynamicalSystem` class is the foundational abstraction in pykal for modeling any control system component. This notebook demonstrates all potential usages of the DynamicalSystem module through progressively complex examples.

## Overview

A `DynamicalSystem` encapsulates:
- **State evolution function** `f`: How the internal state changes over time
- **Observation function** `h`: How we observe or extract information from the state
- **State name**: A string identifying where the state is stored in the parameter dictionary

The power of this abstraction lies in its flexibility and composability. We'll explore:

1. **Creation Patterns**: Stateless systems, stateful systems, default behaviors
2. **Stepping Modes**: Single step, simulation loops, state tracking
3. **Parameter Binding**: How `_smart_call` enables flexible function signatures
4. **Composition**: Chaining multiple systems together
5. **Dynamic Modification**: Changing `f` and `h` at runtime
6. **Real-World Examples**: Sensors, plants, observers, controllers

Let's begin.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from pykal import DynamicalSystem

## Part 1: Creation Patterns

The DynamicalSystem constructor accepts three optional parameters: `f`, `h`, and `state_name`. Different combinations enable different usage patterns.

### 1.1 Stateless System (h only)

A stateless system has no internal state evolution—it simply transforms inputs to outputs. This is useful for:
- Pure mathematical transformations
- Memoryless sensors
- Coordinate transformations
- Setpoint generators with external state

In [ ]:
# Example: Nonlinear sensor that squares its input
def sensor_h(raw_signal: float) -> float:
    """Quadratic sensor response"""
    return raw_signal**2


sensor = DynamicalSystem(h=sensor_h)

# Since there's no f, we don't need state_name
print(f"sensor.f = {sensor.f}")
print(f"sensor.state_name = {sensor.state_name}")

# Step just calls h
output = sensor.step(param_dict={"raw_signal": 3.0})
print(f"\nSensor output: {output}")

### 1.2 Stateful System (f + h + state_name)

A stateful system maintains internal memory and evolves over time. This is the most common pattern for:
- Dynamical systems with physics
- Controllers with integral/derivative terms
- Observers/estimators
- Any system with memory

In [ ]:
# Example: Simple integrator (accumulator)
def integrator_f(state: float, input_signal: float, dt: float) -> float:
    """Integrate input signal over time"""
    return state + input_signal * dt


def integrator_h(state: float) -> float:
    """Output the accumulated integral"""
    return state


integrator = DynamicalSystem(
    f=integrator_f, h=integrator_h, state_name="integral_state"
)

# IMPORTANT: If you provide f, you MUST provide state_name
try:
    bad_system = DynamicalSystem(f=integrator_f, h=integrator_h)  # Missing state_name!
except ValueError as e:
    print(f"Error (expected): {e}")

print(f"\nintegrator.f = {integrator.f}")
print(f"integrator.state_name = {integrator.state_name}")

### 1.3 Default h Behavior (f only)

If you omit `h`, it defaults to the identity function: `h(state) = state`. This is convenient when the observation is simply the entire state.

In [ ]:
# Example: Counter that increments by 1 each step
def counter_f(count: int) -> int:
    return count + 1


# No h provided → defaults to identity
counter = DynamicalSystem(f=counter_f, state_name="count")

# h is automatically set to identity function
print(f"counter.h is default identity: {counter.h}")

# Step returns the state as-is
output = counter.step(param_dict={"count": 0})
print(f"Counter output: {output}")

### 1.4 Multivariate State and Observation

States and observations can be vectors, matrices, or tuples. The `h` function can extract/transform portions of the state.

In [ ]:
# Example: 2D position with velocity (only position is observed)
def position_f(state: np.ndarray, dt: float) -> np.ndarray:
    """Constant velocity motion: state = [x, y, vx, vy]"""
    x, y, vx, vy = state
    return np.array([x + vx * dt, y + vy * dt, vx, vy])


def position_h(state: np.ndarray) -> np.ndarray:
    """Observe only position (first 2 elements)"""
    return state[:2]


moving_object = DynamicalSystem(f=position_f, h=position_h, state_name="pose_state")

# Initial state: at (0, 0) moving at (1, 0.5) m/s
state = np.array([0.0, 0.0, 1.0, 0.5])

# After 1 second
observation = moving_object.step(param_dict={"pose_state": state, "dt": 1.0})
print(f"Observed position: {observation}")

# Get both state and observation
new_state, observation = moving_object.step(
    param_dict={"pose_state": state, "dt": 1.0}, return_state=True
)
print(f"Full state: {new_state}")
print(f"Observation (position only): {observation}")

## Part 2: Stepping Modes

The `step()` method executes one iteration of the dynamical system. It has two modes controlled by the `return_state` parameter.

### 2.1 Basic Step (return_state=False)

By default, `step()` returns only the observation `h(...)`.

In [ ]:
# Simple exponential decay: x_{k+1} = alpha * x_k
def decay_f(x: float, alpha: float) -> float:
    return alpha * x


decay_system = DynamicalSystem(f=decay_f, state_name="x")

# Step once
observation = decay_system.step(param_dict={"x": 10.0, "alpha": 0.9})
print(f"Observation after 1 step: {observation}")

# Note: We get the NEXT state as observation (because h is identity by default)
# But we don't have easy access to it for the next step

### 2.2 Step with State Return (return_state=True)

When simulating, you almost always want `return_state=True` to get both the next state and the observation.

In [ ]:
# Same decay system
x = 10.0
alpha = 0.9

# Step and get both state and observation
x_next, y = decay_system.step(param_dict={"x": x, "alpha": alpha}, return_state=True)

print(f"Current state: x = {x}")
print(f"Next state: x_next = {x_next}")
print(f"Observation: y = {y}")
print(f"(Since h is identity, x_next == y: {x_next == y})")

### 2.3 Simulation Loop Pattern

The standard pattern for simulating a dynamical system over time.

In [ ]:
# Simulate exponential decay for 50 steps
x = 10.0
alpha = 0.9
steps = 50

history = []

for k in range(steps):
    history.append(x)
    x, y = decay_system.step(param_dict={"x": x, "alpha": alpha}, return_state=True)

# Visualize
plt.figure(figsize=(10, 4))
plt.plot(history, "b-", linewidth=2, label=f"x(k), α={alpha}")
plt.xlabel("Time Step (k)", fontsize=12)
plt.ylabel("State Value", fontsize=12)
plt.title("Exponential Decay System", fontsize=14, fontweight="bold")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

### 2.4 Stateless System Stepping

If `f` is None (stateless system), `step()` simply calls `h` directly.

In [ ]:
# Stateless gain system: y = K * u
def gain_h(u: float, K: float) -> float:
    return K * u


gain_system = DynamicalSystem(h=gain_h)

# No state evolution, just transformation
output = gain_system.step(param_dict={"u": 5.0, "K": 2.0})
print(f"Gain system output: {output}")

# return_state doesn't make sense here (no state!)
# It will just return (h(...), h(...)) - same value twice
result = gain_system.step(param_dict={"u": 5.0, "K": 2.0}, return_state=True)
print(f"With return_state=True: {result}")

## Part 3: Smart Parameter Binding

The `_smart_call` method inspects function signatures and automatically binds available parameters from the `param_dict`. This enables flexible function definitions.

### 3.1 Positional and Keyword Arguments

Functions can use any combination of positional and keyword arguments.

In [ ]:
# Different signature styles
def style_1(x: float, u: float) -> float:
    """Positional or keyword arguments"""
    return x + u


def style_2(x: float, *, u: float) -> float:
    """Keyword-only argument (u must be named)"""
    return x + u


def style_3(x: float, u: float, dt: float = 0.1) -> float:
    """With default value"""
    return x + u * dt


# All work with smart_call
sys1 = DynamicalSystem(f=style_1, state_name="x")
sys2 = DynamicalSystem(f=style_2, state_name="x")
sys3 = DynamicalSystem(f=style_3, state_name="x")

params = {"x": 1.0, "u": 2.0, "dt": 0.5}

print(f"Style 1 result: {sys1.step(param_dict=params, return_state=True)[0]}")
print(f"Style 2 result: {sys2.step(param_dict=params, return_state=True)[0]}")
print(f"Style 3 result: {sys3.step(param_dict=params, return_state=True)[0]}")

# If we omit dt, style_3 uses default
params_no_dt = {"x": 1.0, "u": 2.0}
print(
    f"Style 3 with default dt: {sys3.step(param_dict=params_no_dt, return_state=True)[0]}"
)

### 3.2 Using **kwargs for Flexibility

Functions with `**kwargs` can accept arbitrary parameters from the param_dict.

In [ ]:
# Flexible function that accepts any parameters
def flexible_f(x: float, **kwargs) -> float:
    """Use any available parameters"""
    # Extract what we need, with defaults
    a = kwargs.get("a", 1.0)
    b = kwargs.get("b", 0.0)
    c = kwargs.get("c", 0.0)
    return a * x + b * x**2 + c


flexible_sys = DynamicalSystem(f=flexible_f, state_name="x")

# Can provide different parameter combinations
result1 = flexible_sys.step(param_dict={"x": 2.0, "a": 3.0}, return_state=True)[0]
result2 = flexible_sys.step(
    param_dict={"x": 2.0, "a": 3.0, "b": 0.5}, return_state=True
)[0]
result3 = flexible_sys.step(
    param_dict={"x": 2.0, "a": 3.0, "b": 0.5, "c": 1.0}, return_state=True
)[0]

print(f"Result with a only: {result1}")
print(f"Result with a, b: {result2}")
print(f"Result with a, b, c: {result3}")

### 3.3 Parameter Dictionary Sharing

Multiple systems can share a common parameter dictionary. The smart_call mechanism extracts only what each function needs.

In [ ]:
# System 1 needs only x and a
def sys1_f(x: float, a: float) -> float:
    return a * x


# System 2 needs only y and b
def sys2_f(y: float, b: float) -> float:
    return b * y


system_a = DynamicalSystem(f=sys1_f, state_name="x")
system_b = DynamicalSystem(f=sys2_f, state_name="y")

# Shared parameter dictionary with all parameters
shared_params = {
    "x": 2.0,
    "y": 3.0,
    "a": 1.5,
    "b": 2.0,
    "unused_param": 999,  # Ignored by both systems
}

result_a = system_a.step(param_dict=shared_params, return_state=True)[0]
result_b = system_b.step(param_dict=shared_params, return_state=True)[0]

print(f"System A extracts (x, a): {result_a}")
print(f"System B extracts (y, b): {result_b}")

## Part 4: System Composition

The real power of DynamicalSystem emerges when composing multiple systems together. Outputs from one system become inputs to another.

### 4.1 Serial Composition (Chain)

Example: Sensor → Filter → Actuator pipeline

In [ ]:
# System 1: Noisy sensor
def sensor_h_noisy(true_value: float, noise_std: float, rng) -> float:
    return true_value + rng.normal(0, noise_std)


# System 2: Exponential filter (low-pass)
def filter_f(filtered_value: float, measurement: float, alpha: float) -> float:
    """Exponential smoothing: filtered = alpha * measurement + (1-alpha) * filtered_old"""
    return alpha * measurement + (1 - alpha) * filtered_value


# System 3: Actuator with saturation
def actuator_h(filtered_value: float, saturation_limit: float) -> float:
    return np.clip(filtered_value, -saturation_limit, saturation_limit)


sensor_sys = DynamicalSystem(h=sensor_h_noisy)
filter_sys = DynamicalSystem(f=filter_f, state_name="filtered_value")
actuator_sys = DynamicalSystem(h=actuator_h)

# Simulate pipeline
true_signal = np.sin(np.linspace(0, 4 * np.pi, 100))
rng = np.random.default_rng(seed=42)
noise_std = 0.3
alpha = 0.2
saturation_limit = 0.8

filtered_value = 0.0  # Initial filter state
measurements = []
filtered_values = []
actuator_outputs = []

for true_value in true_signal:
    # Step 1: Noisy sensor measurement
    measurement = sensor_sys.step(
        param_dict={"true_value": true_value, "noise_std": noise_std, "rng": rng}
    )

    # Step 2: Filter the measurement
    filtered_value, filtered_out = filter_sys.step(
        param_dict={
            "filtered_value": filtered_value,
            "measurement": measurement,
            "alpha": alpha,
        },
        return_state=True,
    )

    # Step 3: Actuator with saturation
    actuator_output = actuator_sys.step(
        param_dict={
            "filtered_value": filtered_value,
            "saturation_limit": saturation_limit,
        }
    )

    measurements.append(measurement)
    filtered_values.append(filtered_value)
    actuator_outputs.append(actuator_output)

# Visualize pipeline
fig, axs = plt.subplots(3, 1, figsize=(12, 8))

axs[0].plot(true_signal, "k-", linewidth=2, label="True Signal", alpha=0.7)
axs[0].scatter(
    range(len(measurements)),
    measurements,
    c="red",
    s=5,
    alpha=0.3,
    label="Noisy Measurements",
)
axs[0].set_ylabel("Value", fontsize=12)
axs[0].set_title("Sensor: Noisy Measurements", fontsize=14, fontweight="bold")
axs[0].legend()
axs[0].grid(True, alpha=0.3)

axs[1].plot(true_signal, "k--", linewidth=1, label="True Signal", alpha=0.5)
axs[1].plot(filtered_values, "b-", linewidth=2, label=f"Filtered (α={alpha})")
axs[1].set_ylabel("Value", fontsize=12)
axs[1].set_title("Filter: Exponential Smoothing", fontsize=14, fontweight="bold")
axs[1].legend()
axs[1].grid(True, alpha=0.3)

axs[2].plot(filtered_values, "b--", linewidth=1, label="Filtered", alpha=0.5)
axs[2].plot(
    actuator_outputs,
    "g-",
    linewidth=2,
    label=f"Actuator Output (±{saturation_limit} limit)",
)
axs[2].axhline(y=saturation_limit, color="r", linestyle=":", alpha=0.5)
axs[2].axhline(y=-saturation_limit, color="r", linestyle=":", alpha=0.5)
axs[2].set_xlabel("Time Step", fontsize=12)
axs[2].set_ylabel("Value", fontsize=12)
axs[2].set_title("Actuator: With Saturation", fontsize=14, fontweight="bold")
axs[2].legend()
axs[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Pipeline: True Signal → Noisy Sensor → Exponential Filter → Saturating Actuator")

### 4.2 Feedback Composition

Example: Controller → Plant → Sensor → back to Controller

In [ ]:
# Simple proportional controller
def controller_h(setpoint: float, measurement: float, Kp: float) -> float:
    """Proportional control: u = Kp * (setpoint - measurement)"""
    return Kp * (setpoint - measurement)


# First-order plant: dx/dt = -a*x + b*u
def plant_f(x: float, u: float, a: float, b: float, dt: float) -> float:
    return x + dt * (-a * x + b * u)


# Noisy sensor
def sensor_h_feedback(x: float, noise_std: float, rng) -> float:
    return x + rng.normal(0, noise_std)


controller = DynamicalSystem(h=controller_h)
plant = DynamicalSystem(f=plant_f, state_name="x")
sensor = DynamicalSystem(h=sensor_h_feedback)

# Simulation parameters
setpoint = 5.0
Kp = 2.0
a = 1.0
b = 1.0
dt = 0.1
noise_std = 0.1
rng = np.random.default_rng(seed=43)

# Initial state
x = 0.0

# Storage
time_steps = np.arange(0, 10, dt)
states = []
measurements_fb = []
controls = []

for t in time_steps:
    # Sensor measures plant state
    measurement = sensor.step(param_dict={"x": x, "noise_std": noise_std, "rng": rng})

    # Controller generates control input based on measurement
    u = controller.step(
        param_dict={"setpoint": setpoint, "measurement": measurement, "Kp": Kp}
    )

    # Plant evolves under control input
    x, y = plant.step(
        param_dict={"x": x, "u": u, "a": a, "b": b, "dt": dt}, return_state=True
    )

    states.append(x)
    measurements_fb.append(measurement)
    controls.append(u)

# Visualize feedback loop
fig, axs = plt.subplots(2, 1, figsize=(12, 7))

axs[0].axhline(y=setpoint, color="k", linestyle="--", linewidth=2, label="Setpoint")
axs[0].plot(time_steps, states, "b-", linewidth=2, label="Plant State (x)", alpha=0.8)
axs[0].scatter(
    time_steps[::5],
    measurements_fb[::5],
    c="red",
    s=20,
    alpha=0.5,
    label="Noisy Measurements",
)
axs[0].set_ylabel("State Value", fontsize=12)
axs[0].set_title(
    f"Feedback Control System (Kp={Kp}, noise_std={noise_std})",
    fontsize=14,
    fontweight="bold",
)
axs[0].legend()
axs[0].grid(True, alpha=0.3)

axs[1].plot(time_steps, controls, "g-", linewidth=2, label="Control Input (u)")
axs[1].set_xlabel("Time (s)", fontsize=12)
axs[1].set_ylabel("Control Input", fontsize=12)
axs[1].set_title("Controller Output", fontsize=14, fontweight="bold")
axs[1].legend()
axs[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Final state: {states[-1]:.3f} (setpoint: {setpoint})")
print(f"Steady-state error: {setpoint - states[-1]:.3f}")

### 4.3 Parallel Composition

Example: Multiple sensors observing the same plant state

In [ ]:
# Plant: Simple harmonic oscillator
def oscillator_f(state: np.ndarray, dt: float) -> np.ndarray:
    """State = [position, velocity]"""
    pos, vel = state
    omega = 2.0  # Natural frequency
    return np.array([pos + vel * dt, vel - omega**2 * pos * dt])


# Three different sensors
def position_sensor_h(state: np.ndarray, noise_std: float, rng) -> float:
    """Measures position with noise"""
    return state[0] + rng.normal(0, noise_std)


def velocity_sensor_h(state: np.ndarray, noise_std: float, rng) -> float:
    """Measures velocity with noise"""
    return state[1] + rng.normal(0, noise_std)


def energy_sensor_h(state: np.ndarray, noise_std: float, rng) -> float:
    """Estimates energy: E = 0.5 * (v^2 + omega^2 * x^2)"""
    omega = 2.0
    pos, vel = state
    energy = 0.5 * (vel**2 + omega**2 * pos**2)
    return energy + rng.normal(0, noise_std)


oscillator = DynamicalSystem(f=oscillator_f, state_name="state")
pos_sensor = DynamicalSystem(h=position_sensor_h)
vel_sensor = DynamicalSystem(h=velocity_sensor_h)
energy_sensor = DynamicalSystem(h=energy_sensor_h)

# Simulate with all three sensors
state = np.array([1.0, 0.0])  # Initial: displaced, at rest
dt = 0.05
time_steps_osc = np.arange(0, 10, dt)
rng_osc = np.random.default_rng(seed=44)

states_osc = []
pos_measurements = []
vel_measurements = []
energy_measurements = []

for t in time_steps_osc:
    # All sensors measure current state in parallel
    pos_meas = pos_sensor.step(
        param_dict={"state": state, "noise_std": 0.05, "rng": rng_osc}
    )
    vel_meas = vel_sensor.step(
        param_dict={"state": state, "noise_std": 0.1, "rng": rng_osc}
    )
    energy_meas = energy_sensor.step(
        param_dict={"state": state, "noise_std": 0.2, "rng": rng_osc}
    )

    states_osc.append(state.copy())
    pos_measurements.append(pos_meas)
    vel_measurements.append(vel_meas)
    energy_measurements.append(energy_meas)

    # Plant evolves
    state, _ = oscillator.step(param_dict={"state": state, "dt": dt}, return_state=True)

states_osc = np.array(states_osc)

# Visualize parallel sensors
fig, axs = plt.subplots(3, 1, figsize=(12, 9))

axs[0].plot(
    time_steps_osc,
    states_osc[:, 0],
    "b-",
    linewidth=2,
    label="True Position",
    alpha=0.7,
)
axs[0].scatter(
    time_steps_osc[::10],
    pos_measurements[::10],
    c="red",
    s=15,
    alpha=0.5,
    label="Position Sensor",
)
axs[0].set_ylabel("Position", fontsize=12)
axs[0].set_title("Sensor 1: Position Measurement", fontsize=14, fontweight="bold")
axs[0].legend()
axs[0].grid(True, alpha=0.3)

axs[1].plot(
    time_steps_osc,
    states_osc[:, 1],
    "g-",
    linewidth=2,
    label="True Velocity",
    alpha=0.7,
)
axs[1].scatter(
    time_steps_osc[::10],
    vel_measurements[::10],
    c="orange",
    s=15,
    alpha=0.5,
    label="Velocity Sensor",
)
axs[1].set_ylabel("Velocity", fontsize=12)
axs[1].set_title("Sensor 2: Velocity Measurement", fontsize=14, fontweight="bold")
axs[1].legend()
axs[1].grid(True, alpha=0.3)

true_energy = 0.5 * (states_osc[:, 1] ** 2 + 4.0 * states_osc[:, 0] ** 2)
axs[2].plot(
    time_steps_osc, true_energy, "purple", linewidth=2, label="True Energy", alpha=0.7
)
axs[2].scatter(
    time_steps_osc[::10],
    energy_measurements[::10],
    c="cyan",
    s=15,
    alpha=0.5,
    label="Energy Sensor",
)
axs[2].set_xlabel("Time (s)", fontsize=12)
axs[2].set_ylabel("Energy", fontsize=12)
axs[2].set_title("Sensor 3: Energy Measurement", fontsize=14, fontweight="bold")
axs[2].legend()
axs[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Three sensors measuring the same plant state in parallel")

## Part 5: Dynamic Modification

The `f` and `h` functions can be modified at runtime using property setters. This enables:
- Switching control strategies
- Adaptive systems
- Fault-tolerant architectures
- Testing different models

### 5.1 Switching f at Runtime

Example: Switching between linear and nonlinear plant models

In [ ]:
# Two different evolution functions
def linear_f(x: float, u: float, dt: float) -> float:
    """Linear dynamics: dx/dt = u"""
    return x + u * dt


def nonlinear_f(x: float, u: float, dt: float) -> float:
    """Nonlinear dynamics: dx/dt = u * (1 - x^2)"""
    return x + u * (1 - x**2) * dt


# Create system with linear model
adaptive_system = DynamicalSystem(f=linear_f, state_name="x")

# Simulate with constant input
x = 0.0
u = 1.0
dt = 0.1
switch_time = 50

linear_phase = []
nonlinear_phase = []

# Linear phase
for k in range(switch_time):
    linear_phase.append(x)
    x, _ = adaptive_system.step(
        param_dict={"x": x, "u": u, "dt": dt}, return_state=True
    )

# Switch to nonlinear model
adaptive_system.f = nonlinear_f
print(f"Switched from linear to nonlinear model at step {switch_time}")

# Nonlinear phase
for k in range(switch_time):
    nonlinear_phase.append(x)
    x, _ = adaptive_system.step(
        param_dict={"x": x, "u": u, "dt": dt}, return_state=True
    )

# Visualize
plt.figure(figsize=(12, 5))
plt.plot(linear_phase, "b-", linewidth=2, label="Linear Model")
plt.plot(
    range(switch_time, switch_time + len(nonlinear_phase)),
    nonlinear_phase,
    "r-",
    linewidth=2,
    label="Nonlinear Model",
)
plt.axvline(x=switch_time, color="k", linestyle="--", alpha=0.5, label="Model Switch")
plt.xlabel("Time Step", fontsize=12)
plt.ylabel("State (x)", fontsize=12)
plt.title("Dynamic Model Switching", fontsize=14, fontweight="bold")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Note: Nonlinear model saturates as x approaches ±1")

### 5.2 Switching h at Runtime

Example: Sensor failure recovery—switching from high-precision to backup sensor

In [ ]:
# Plant with 2D state [position, velocity]
def plant_2d_f(state: np.ndarray, u: float, dt: float) -> np.ndarray:
    pos, vel = state
    return np.array([pos + vel * dt, vel + u * dt])


# High-precision sensor (measures both position and velocity)
def high_precision_h(state: np.ndarray, noise_std: float, rng) -> np.ndarray:
    return state + rng.normal(0, noise_std, size=2)


# Backup sensor (position only, higher noise)
def backup_h(state: np.ndarray, noise_std: float, rng) -> float:
    return state[0] + rng.normal(0, noise_std * 5)  # 5x more noise


plant_2d = DynamicalSystem(f=plant_2d_f, state_name="state")
sensor_sys = DynamicalSystem(h=high_precision_h)

# Simulate
state = np.array([0.0, 1.0])  # Start at origin with velocity = 1
u = 0.1  # Small acceleration
dt = 0.1
rng_sensor = np.random.default_rng(seed=45)
failure_time = 50

measurements_pre_failure = []
measurements_post_failure = []
true_states = []

for k in range(100):
    # Simulate sensor failure at step 50
    if k == failure_time:
        sensor_sys.h = backup_h
        print(f"Sensor failure at step {failure_time}! Switched to backup sensor.")

    # Measure
    measurement = sensor_sys.step(
        param_dict={"state": state, "noise_std": 0.02, "rng": rng_sensor}
    )

    # Store
    true_states.append(state.copy())
    if k < failure_time:
        measurements_pre_failure.append(measurement)
    else:
        measurements_post_failure.append(measurement)

    # Plant evolves
    state, _ = plant_2d.step(
        param_dict={"state": state, "u": u, "dt": dt}, return_state=True
    )

true_states = np.array(true_states)
measurements_pre_failure = np.array(measurements_pre_failure)

# Visualize
plt.figure(figsize=(12, 5))
plt.plot(true_states[:, 0], "k-", linewidth=2, label="True Position", alpha=0.7)

# Pre-failure: 2D measurements (plot position component)
plt.scatter(
    range(len(measurements_pre_failure)),
    measurements_pre_failure[:, 0],
    c="blue",
    s=20,
    alpha=0.5,
    label="High-Precision Sensor",
)

# Post-failure: 1D measurements (position only, higher noise)
plt.scatter(
    range(failure_time, 100),
    measurements_post_failure,
    c="red",
    s=20,
    alpha=0.5,
    label="Backup Sensor (degraded)",
)

plt.axvline(
    x=failure_time, color="orange", linestyle="--", linewidth=2, label="Sensor Failure"
)
plt.xlabel("Time Step", fontsize=12)
plt.ylabel("Position", fontsize=12)
plt.title("Sensor Fault Tolerance: Dynamic h Switching", fontsize=14, fontweight="bold")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Notice increased noise after switching to backup sensor")

### 5.3 Modifying state_name

You can also change the `state_name` property to redirect where state is stored in the param_dict.

In [ ]:
# Simple counter
def increment_f(val: int) -> int:
    return val + 1


counter_sys = DynamicalSystem(f=increment_f, state_name="counter_a")

# Initial param dict with two counters
params = {"counter_a": 0, "counter_b": 100}

# Step using counter_a
params["counter_a"], _ = counter_sys.step(param_dict=params, return_state=True)
print(f"After step with counter_a: {params}")

# Switch to counter_b
counter_sys.state_name = "counter_b"

# Step using counter_b
params["counter_b"], _ = counter_sys.step(param_dict=params, return_state=True)
print(f"After step with counter_b: {params}")

print("\nSame system, different state storage locations")

## Part 6: Real-World Control System Examples

Let's combine everything we've learned into realistic control system scenarios.

### 6.1 PID Controller with Anti-Windup

A complete PID implementation showing stateful controller design.

In [ ]:
# PID state: (integral, previous_error)
def pid_f(pid_state: tuple, error: float, dt: float, integral_limit: float) -> tuple:
    """
    PID controller with anti-windup.
    State: (integral, prev_error)
    """
    integral, prev_error = pid_state

    # Update integral with anti-windup
    integral = np.clip(integral + error * dt, -integral_limit, integral_limit)

    return (integral, error)


def pid_h(
    pid_state: tuple, error: float, Kp: float, Ki: float, Kd: float, dt: float
) -> float:
    """
    Compute PID control output.
    """
    integral, prev_error = pid_state
    derivative = (error - prev_error) / dt
    return Kp * error + Ki * integral + Kd * derivative


pid_controller = DynamicalSystem(f=pid_f, h=pid_h, state_name="pid_state")


# Plant: simple integrator (velocity control)
def velocity_plant_f(velocity: float, force: float, damping: float, dt: float) -> float:
    return velocity + dt * (force - damping * velocity)


velocity_plant = DynamicalSystem(f=velocity_plant_f, state_name="velocity")

# Simulation
setpoint = 10.0
Kp = 1.0
Ki = 0.5
Kd = 0.2
damping = 0.5
dt = 0.05
integral_limit = 20.0

pid_state = (0.0, 0.0)  # (integral, prev_error)
velocity = 0.0

time_pid = np.arange(0, 20, dt)
velocities = []
forces = []
errors_pid = []

for t in time_pid:
    error = setpoint - velocity
    errors_pid.append(error)

    # PID step
    pid_state, force = pid_controller.step(
        param_dict={
            "pid_state": pid_state,
            "error": error,
            "dt": dt,
            "integral_limit": integral_limit,
            "Kp": Kp,
            "Ki": Ki,
            "Kd": Kd,
        },
        return_state=True,
    )

    forces.append(force)
    velocities.append(velocity)

    # Plant step
    velocity, _ = velocity_plant.step(
        param_dict={"velocity": velocity, "force": force, "damping": damping, "dt": dt},
        return_state=True,
    )

# Visualize
fig, axs = plt.subplots(3, 1, figsize=(12, 9))

axs[0].axhline(y=setpoint, color="k", linestyle="--", linewidth=2, label="Setpoint")
axs[0].plot(time_pid, velocities, "b-", linewidth=2, label="Velocity")
axs[0].set_ylabel("Velocity", fontsize=12)
axs[0].set_title(
    f"PID Velocity Control (Kp={Kp}, Ki={Ki}, Kd={Kd})", fontsize=14, fontweight="bold"
)
axs[0].legend()
axs[0].grid(True, alpha=0.3)

axs[1].plot(time_pid, forces, "g-", linewidth=2, label="Control Force")
axs[1].set_ylabel("Force", fontsize=12)
axs[1].set_title("PID Output", fontsize=14, fontweight="bold")
axs[1].legend()
axs[1].grid(True, alpha=0.3)

axs[2].plot(time_pid, errors_pid, "m-", linewidth=2, label="Tracking Error")
axs[2].axhline(y=0, color="k", linestyle=":", alpha=0.5)
axs[2].set_xlabel("Time (s)", fontsize=12)
axs[2].set_ylabel("Error", fontsize=12)
axs[2].set_title("Tracking Error", fontsize=14, fontweight="bold")
axs[2].legend()
axs[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Final velocity: {velocities[-1]:.3f} (setpoint: {setpoint})")

### 6.2 Observer-Based Control (Separation Principle)

Plant with hidden state → Observer estimates state → Controller uses estimate

In [ ]:
# Hidden state plant: 2D [position, velocity]
def hidden_plant_f(state: np.ndarray, u: float, dt: float) -> np.ndarray:
    pos, vel = state
    return np.array([pos + vel * dt, vel + u * dt])


def hidden_plant_h(state: np.ndarray, noise_std: float, rng) -> float:
    """Only position is measured, with noise"""
    return state[0] + rng.normal(0, noise_std)


# Simple Luenberger observer
def observer_f(
    state_estimate: np.ndarray, u: float, measurement: float, L: np.ndarray, dt: float
) -> np.ndarray:
    """
    Observer dynamics with correction term.
    L is the observer gain vector [L_pos, L_vel]
    """
    pos_est, vel_est = state_estimate

    # Prediction
    pos_pred = pos_est + vel_est * dt
    vel_pred = vel_est + u * dt

    # Correction based on position measurement
    innovation = measurement - pos_pred
    pos_est = pos_pred + L[0] * innovation
    vel_est = vel_pred + L[1] * innovation

    return np.array([pos_est, vel_est])


# State feedback controller: u = -K * x_estimate
def state_feedback_h(
    state_estimate: np.ndarray, reference: float, K: np.ndarray
) -> float:
    """State feedback: u = -K * (x - r)"""
    error = state_estimate - np.array([reference, 0.0])
    return -K @ error


hidden_plant = DynamicalSystem(
    f=hidden_plant_f, h=hidden_plant_h, state_name="true_state"
)
observer = DynamicalSystem(f=observer_f, state_name="state_estimate")
controller_sf = DynamicalSystem(h=state_feedback_h)

# Simulation
true_state = np.array([0.0, 0.0])
state_estimate = np.array([0.0, 0.0])  # Start with wrong estimate
reference = 5.0
K = np.array([2.0, 3.0])  # Controller gains
L = np.array([5.0, 8.0])  # Observer gains
dt = 0.05
noise_std = 0.1
rng_obs = np.random.default_rng(seed=46)

time_obs = np.arange(0, 15, dt)
true_states_obs = []
state_estimates = []
controls_obs = []
measurements_obs = []

for t in time_obs:
    # Plant measurement (position only)
    measurement = hidden_plant.h(true_state, noise_std=noise_std, rng=rng_obs)

    # Controller generates input based on state estimate
    u = controller_sf.step(
        param_dict={"state_estimate": state_estimate, "reference": reference, "K": K}
    )

    # Observer updates estimate
    state_estimate, _ = observer.step(
        param_dict={
            "state_estimate": state_estimate,
            "u": u,
            "measurement": measurement,
            "L": L,
            "dt": dt,
        },
        return_state=True,
    )

    # Plant evolves
    true_state, _ = hidden_plant.step(
        param_dict={
            "true_state": true_state,
            "u": u,
            "dt": dt,
            "noise_std": noise_std,
            "rng": rng_obs,
        },
        return_state=True,
    )

    true_states_obs.append(true_state.copy())
    state_estimates.append(state_estimate.copy())
    controls_obs.append(u)
    measurements_obs.append(measurement)

true_states_obs = np.array(true_states_obs)
state_estimates = np.array(state_estimates)

# Visualize observer-based control
fig, axs = plt.subplots(3, 1, figsize=(12, 10))

axs[0].axhline(y=reference, color="k", linestyle="--", linewidth=2, label="Reference")
axs[0].plot(
    time_obs, true_states_obs[:, 0], "b-", linewidth=2, label="True Position", alpha=0.8
)
axs[0].plot(
    time_obs,
    state_estimates[:, 0],
    "r--",
    linewidth=2,
    label="Estimated Position",
    alpha=0.8,
)
axs[0].scatter(
    time_obs[::10],
    measurements_obs[::10],
    c="gray",
    s=20,
    alpha=0.3,
    label="Measurements",
)
axs[0].set_ylabel("Position", fontsize=12)
axs[0].set_title("Observer-Based Control: Position", fontsize=14, fontweight="bold")
axs[0].legend()
axs[0].grid(True, alpha=0.3)

axs[1].plot(
    time_obs, true_states_obs[:, 1], "b-", linewidth=2, label="True Velocity", alpha=0.8
)
axs[1].plot(
    time_obs,
    state_estimates[:, 1],
    "r--",
    linewidth=2,
    label="Estimated Velocity",
    alpha=0.8,
)
axs[1].set_ylabel("Velocity", fontsize=12)
axs[1].set_title(
    "Velocity (Unmeasured, Estimated by Observer)", fontsize=14, fontweight="bold"
)
axs[1].legend()
axs[1].grid(True, alpha=0.3)

axs[2].plot(time_obs, controls_obs, "g-", linewidth=2, label="Control Input")
axs[2].set_xlabel("Time (s)", fontsize=12)
axs[2].set_ylabel("Control Input", fontsize=12)
axs[2].set_title("Controller Output", fontsize=14, fontweight="bold")
axs[2].legend()
axs[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Observer estimates unmeasured velocity from position measurements only!")

## Part 7: Wrapping Systems in Callbacks

While `DynamicalSystem` provides powerful abstractions for modeling control components, real-world deployment (especially to ROS2 nodes) requires wrapping these systems in **callbacks**. Callbacks provide:

- **Simplicity**: Hide complex state management behind a clean function interface
- **Composability**: Combine multiple DynamicalSystems into a single callable unit
- **ROS2 Compatibility**: Match the signature expected by ROSNode wrappers

A ROS-compatible callback has the signature:
```python
def callback(tk: float, **kwargs) -> dict:
    # tk: current time in seconds
    # **kwargs: input topics as keyword arguments
    # returns: dict mapping output names to values
```

This part demonstrates how to wrap DynamicalSystems in callbacks for deployment.

### 7.1 Basic Callback Wrapper

The simplest callback wraps a single stateless DynamicalSystem.

In [ ]:
# Example: Proportional controller as a callback
def controller_h_simple(setpoint: float, measurement: float, Kp: float) -> float:
    """P controller: u = Kp * (setpoint - measurement)"""
    return Kp * (setpoint - measurement)

# Wrap in DynamicalSystem
controller_sys = DynamicalSystem(h=controller_h_simple)

# Create a callback wrapper
def controller_callback(tk: float, setpoint: float, measurement: float) -> dict:
    """
    Callback signature for ROS deployment.
    
    Args:
        tk: Current time in seconds (required by ROS)
        setpoint: Desired value
        measurement: Measured value from sensor
    
    Returns:
        Dictionary with output 'control_effort'
    """
    Kp = 2.0  # Controller gain
    
    # Step the dynamical system
    u = controller_sys.step(param_dict={
        "setpoint": setpoint,
        "measurement": measurement,
        "Kp": Kp
    })
    
    # Return as dict (ROS convention)
    return {"control_effort": u}

# Test the callback
result = controller_callback(tk=0.0, setpoint=10.0, measurement=3.0)
print(f"Controller callback output: {result}")
print(f"Control effort: {result['control_effort']}")

### 7.2 Stateful Callback with Closure

For stateful systems, we use closure to maintain state between callback invocations.

In [ ]:
# Integrator system (stateful)
def make_integrator_callback(initial_state: float = 0.0, dt: float = 0.1):
    """
    Factory function that creates a stateful integrator callback.
    Uses closure to maintain state between calls.
    """
    # Define the integrator DynamicalSystem
    def integrator_f_cb(integral: float, input_signal: float, dt: float) -> float:
        return integral + input_signal * dt
    
    integrator_sys = DynamicalSystem(f=integrator_f_cb, state_name="integral")
    
    # State stored in closure
    state = {"integral": initial_state}
    
    def callback(tk: float, input_signal: float) -> dict:
        """
        Integrator callback.
        
        Args:
            tk: Current time in seconds
            input_signal: Signal to integrate
        
        Returns:
            Dictionary with 'integral' output
        """
        # Step the system and update state
        state["integral"], output = integrator_sys.step(
            param_dict={"integral": state["integral"], "input_signal": input_signal, "dt": dt},
            return_state=True
        )
        
        return {"integral": output}
    
    return callback

# Create integrator callback
integrator_cb = make_integrator_callback(initial_state=0.0, dt=0.1)

# Simulate by calling multiple times
print("Integrating constant input of 1.0:")
for k in range(5):
    result = integrator_cb(tk=k*0.1, input_signal=1.0)
    print(f"  Step {k}: integral = {result['integral']:.2f}")

### 7.3 Composing Multiple Systems in a Callback

The real power emerges when composing multiple DynamicalSystems in a single callback—this represents a complete control loop or processing pipeline.

In [ ]:
def make_filter_controller_callback(Kp: float = 2.0, filter_alpha: float = 0.3, dt: float = 0.1):
    """
    Creates a callback that:
    1. Filters noisy measurements (exponential smoothing)
    2. Applies proportional control to filtered signal
    
    This shows composition: Measurement → Filter → Controller → Control
    """
    # Filter: stateful (exponential smoother)
    def filter_f_comp(filtered: float, measurement: float, alpha: float) -> float:
        return alpha * measurement + (1 - alpha) * filtered
    
    filter_sys = DynamicalSystem(f=filter_f_comp, state_name="filtered")
    
    # Controller: stateless (proportional)
    def controller_h_comp(setpoint: float, filtered: float, Kp: float) -> float:
        return Kp * (setpoint - filtered)
    
    controller_sys = DynamicalSystem(h=controller_h_comp)
    
    # State in closure
    state = {"filtered": 0.0}
    
    def callback(tk: float, setpoint: float, measurement: float) -> dict:
        """
        Composed filter + controller callback.
        
        Args:
            tk: Current time
            setpoint: Desired value
            measurement: Noisy sensor reading
        
        Returns:
            Dict with 'control_effort' and 'filtered_measurement'
        """
        # Step 1: Filter the measurement
        state["filtered"], filtered_value = filter_sys.step(
            param_dict={
                "filtered": state["filtered"],
                "measurement": measurement,
                "alpha": filter_alpha
            },
            return_state=True
        )
        
        # Step 2: Apply control based on filtered value
        control = controller_sys.step(param_dict={
            "setpoint": setpoint,
            "filtered": filtered_value,
            "Kp": Kp
        })
        
        return {
            "control_effort": control,
            "filtered_measurement": filtered_value
        }
    
    return callback

# Create the composed callback
filter_controller = make_filter_controller_callback(Kp=2.0, filter_alpha=0.3)

# Simulate with noisy measurements
print("\nFilter + Controller composition:")
setpoint = 10.0
rng_cb = np.random.default_rng(seed=47)
true_value = 3.0

for k in range(6):
    # Noisy measurement
    noisy_meas = true_value + rng_cb.normal(0, 0.5)
    
    # Call composed callback
    result = filter_controller(tk=k*0.1, setpoint=setpoint, measurement=noisy_meas)
    
    print(f"  Step {k}: measurement={noisy_meas:.2f}, filtered={result['filtered_measurement']:.2f}, control={result['control_effort']:.2f}")

### 7.4 Complete Feedback Loop in a Callback

This example shows a complete observer + controller system wrapped in a single callback, demonstrating how complex control diagrams map to callbacks.

In [ ]:
def make_observer_controller_callback(
    K: np.ndarray = np.array([2.0, 1.5]),  # Controller gains
    L: np.ndarray = np.array([3.0, 5.0]),  # Observer gains
    dt: float = 0.1
):
    """
    Creates a callback implementing observer-based control:
    
    Measurement → Observer → State Estimate → Controller → Control Input
    
    This is a complete control system in a single callback!
    """
    # Observer: estimates [position, velocity] from position measurements
    def observer_f_cb(
        state_est: np.ndarray, 
        measurement: float, 
        u: float, 
        L: np.ndarray, 
        dt: float
    ) -> np.ndarray:
        pos_est, vel_est = state_est
        
        # Predict
        pos_pred = pos_est + vel_est * dt
        vel_pred = vel_est + u * dt
        
        # Correct
        innovation = measurement - pos_pred
        pos_est = pos_pred + L[0] * innovation
        vel_est = vel_pred + L[1] * innovation
        
        return np.array([pos_est, vel_est])
    
    observer_sys = DynamicalSystem(f=observer_f_cb, state_name="state_est")
    
    # State feedback controller
    def controller_h_cb(state_est: np.ndarray, reference: float, K: np.ndarray) -> float:
        error = state_est - np.array([reference, 0.0])
        return -K @ error
    
    controller_sys = DynamicalSystem(h=controller_h_cb)
    
    # State in closure
    state = {
        "state_est": np.array([0.0, 0.0]),
        "u_prev": 0.0  # Need previous control for observer prediction
    }
    
    def callback(tk: float, measurement: float, reference: float) -> dict:
        """
        Observer-based controller callback.
        
        Args:
            tk: Current time
            measurement: Position measurement
            reference: Desired position
        
        Returns:
            Dict with 'control_input', 'position_estimate', 'velocity_estimate'
        """
        # Step 1: Observer updates state estimate
        state["state_est"], state_est = observer_sys.step(
            param_dict={
                "state_est": state["state_est"],
                "measurement": measurement,
                "u": state["u_prev"],
                "L": L,
                "dt": dt
            },
            return_state=True
        )
        
        # Step 2: Controller generates control input
        u = controller_sys.step(param_dict={
            "state_est": state_est,
            "reference": reference,
            "K": K
        })
        
        # Save control for next observer prediction
        state["u_prev"] = u
        
        return {
            "control_input": u,
            "position_estimate": state_est[0],
            "velocity_estimate": state_est[1]
        }
    
    return callback

# Create the observer-controller callback
obs_controller = make_observer_controller_callback(
    K=np.array([2.0, 1.5]),
    L=np.array([3.0, 5.0]),
    dt=0.1
)

# Simulate the callback receiving measurements
print("\nObserver + Controller composition:")
print("Measurement → Observer → Controller → Control Input")
print()

reference = 5.0
for k in range(8):
    # Simulate position measurement (gradually increasing)
    measurement = k * 0.5
    
    # Call the callback
    result = obs_controller(tk=k*0.1, measurement=measurement, reference=reference)
    
    print(f"Step {k}: meas={measurement:.1f} → "
          f"est_pos={result['position_estimate']:.2f}, "
          f"est_vel={result['velocity_estimate']:.2f} → "
          f"control={result['control_input']:.2f}")

### 7.5 Visualizing Callback-Based Control

Let's demonstrate a complete simulation using only the callback interface—this is exactly how it will work when deployed to ROS2!

In [ ]:
# Create a plant to simulate (this would be the real robot in ROS)
def make_plant_simulator(dt: float = 0.1):
    """Simulates a simple 2D plant [position, velocity]"""
    def plant_f_sim(state: np.ndarray, u: float, dt: float) -> np.ndarray:
        pos, vel = state
        return np.array([pos + vel * dt, vel + u * dt])
    
    plant_sys = DynamicalSystem(f=plant_f_sim, state_name="state")
    state = {"state": np.array([0.0, 0.0])}
    
    def plant_callback(tk: float, control_input: float) -> dict:
        """Plant receives control input, returns position measurement"""
        state["state"], _ = plant_sys.step(
            param_dict={"state": state["state"], "u": control_input, "dt": dt},
            return_state=True
        )
        
        # Add measurement noise
        rng_plant = np.random.default_rng(seed=int(tk*1000) % 2**32)
        measurement = state["state"][0] + rng_plant.normal(0, 0.1)
        
        return {
            "measurement": measurement,
            "true_position": state["state"][0],
            "true_velocity": state["state"][1]
        }
    
    return plant_callback

# Create plant and controller callbacks
plant_cb = make_plant_simulator(dt=0.05)
controller_cb = make_observer_controller_callback(
    K=np.array([3.0, 2.0]),
    L=np.array([8.0, 12.0]),
    dt=0.05
)

# Run closed-loop simulation using ONLY callbacks
reference = 10.0
time_vec = np.arange(0, 10, 0.05)

# Storage
measurements_log = []
true_positions = []
estimates = []
controls = []

# Initial control
u = 0.0

for tk in time_vec:
    # Plant responds to control (simulates physical system)
    plant_output = plant_cb(tk=tk, control_input=u)
    
    # Controller processes measurement and generates new control
    controller_output = controller_cb(
        tk=tk,
        measurement=plant_output["measurement"],
        reference=reference
    )
    
    # Update control for next iteration
    u = controller_output["control_input"]
    
    # Log data
    measurements_log.append(plant_output["measurement"])
    true_positions.append(plant_output["true_position"])
    estimates.append(controller_output["position_estimate"])
    controls.append(u)

# Visualize
fig, axs = plt.subplots(2, 1, figsize=(12, 8))

axs[0].axhline(y=reference, color='k', linestyle='--', linewidth=2, label='Reference')
axs[0].plot(time_vec, true_positions, 'b-', linewidth=2, label='True Position', alpha=0.8)
axs[0].plot(time_vec, estimates, 'r--', linewidth=2, label='Estimated Position', alpha=0.8)
axs[0].scatter(time_vec[::20], measurements_log[::20], c='gray', s=10, alpha=0.3, label='Noisy Measurements')
axs[0].set_ylabel('Position', fontsize=12)
axs[0].set_title('Callback-Based Control System (Ready for ROS2!)', fontsize=14, fontweight='bold')
axs[0].legend()
axs[0].grid(True, alpha=0.3)

axs[1].plot(time_vec, controls, 'g-', linewidth=2, label='Control Input')
axs[1].set_xlabel('Time (s)', fontsize=12)
axs[1].set_ylabel('Control Input', fontsize=12)
axs[1].set_title('Controller Output', fontsize=14, fontweight='bold')
axs[1].legend()
axs[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal position: {true_positions[-1]:.2f} (reference: {reference})")
print(f"Final estimate: {estimates[-1]:.2f}")
print(f"\nThis callback is ready to be wrapped in a ROSNode!")

### 7.6 Key Takeaways: Callbacks for ROS Deployment

Wrapping DynamicalSystems in callbacks provides the bridge from theory to deployment:

**Why Callbacks?**
- **ROS2 Compatibility**: ROSNode expects callbacks with signature `(tk, **inputs) -> dict`
- **Encapsulation**: Hide complex state management and composition
- **Testability**: Test entire control loops in pure Python before ROS deployment
- **Modularity**: Swap algorithms by changing the callback factory function

**Design Patterns**:
1. **Stateless callbacks**: Directly wrap stateless DynamicalSystems
2. **Stateful callbacks**: Use closure to maintain state between calls
3. **Factory functions**: `make_X_callback()` pattern for configuration
4. **Composition**: Combine multiple DynamicalSystems in one callback

**Next Steps**:
- The callbacks we created here can be directly passed to `ROSNode()` for deployment
- ROS topics map to callback arguments and return dict keys
- Message conversion happens automatically via `ROS2PY_DEFAULT` and `PY2ROS_DEFAULT`

:::{tip}
When designing control systems:
1. Model components as DynamicalSystems (as shown in Parts 1-6)
2. Wrap compositions in callbacks (as shown in Part 7)
3. Test callbacks in pure Python simulations
4. Deploy to ROS2 using ROSNode wrappers (shown in later tutorials)
:::

See the [ROS Deployment tutorial](../python_to_ros/ros_deployment.ipynb) for examples of wrapping these callbacks in ROSNode for hardware deployment!

## Summary

This notebook demonstrated comprehensive usage of the `DynamicalSystem` class:

### Creation Patterns
- Stateless systems (h only) for transformations and memoryless operations
- Stateful systems (f + h + state_name) for dynamics with memory
- Default h behavior (identity function) for direct state observation
- Multivariate states and observations

### Stepping Modes
- Basic step returning observation only
- Step with state return for simulation loops
- Stateless stepping (direct h call)

### Smart Parameter Binding
- Flexible function signatures (positional, keyword, defaults)
- **kwargs for maximum flexibility
- Shared parameter dictionaries across multiple systems

### System Composition
- Serial composition (sensor → filter → actuator pipelines)
- Feedback composition (controller → plant → sensor → controller)
- Parallel composition (multiple sensors observing same plant)

### Dynamic Modification
- Switching f at runtime (adaptive control, fault tolerance)
- Switching h at runtime (sensor reconfiguration)
- Modifying state_name (redirecting state storage)

### Real-World Examples
- PID controller with anti-windup
- Observer-based control (separation principle)
- Sensor fusion and fault tolerance

:::{note}
The `DynamicalSystem` abstraction enables composable, testable, and flexible control system design. By modeling all components (plants, sensors, controllers, observers) as dynamical systems with `f` and `h` functions, we can:
- Test components in isolation
- Compose systems hierarchically
- Swap implementations at runtime
- Share parameter dictionaries seamlessly
- Deploy the same code from simulation to hardware
:::

For complete examples integrating DynamicalSystem with ROS2 deployment, see:
- [Car Cruise Control](car_cruise_control.ipynb)
- [TurtleBot State Estimation](turtlebot_state_estimation.ipynb)
- [Crazyflie Sensor Fusion](crazyflie_sensor_fusion.ipynb)

[← Modules](../../../getting_started/theory_to_python/modules.rst)